In [1]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os, os.path, pickle, sys

import matplotlib
import matplotlib.pyplot as plt

In [2]:
dpi_, map_dpi_ = 300, 300

In [3]:
os.getcwd()

'd:\\github\\wheat_transformers\\notebooks_drivers\\01_pre-process'

In [ ]:
# data_base = "/Users/hn/Documents/01_research_data/Ehsan/wheat/Data/"
# Define the base directory relative to your current working directory
dir_base = "./../../../01_research_data/Ehsan/wheat/"
data_base = "D:/01_research_data/Ehsan/Wheat/Data/"
#data_base = dir_base + "Data/"
raw_dir = os.path.join(data_base, "00_raw_by_location/")
# raw_dir = os.path.abspath(raw_dir)

In [5]:
fullName_units = pd.read_csv(data_base + "inputs_column_names_v1.csv")
fullName_units

,variable_name,variable_short,source,reference,unit
0,precip_mmday,Pr,gridMET,marszalek2022prediction,mmday
1,max_temp_c,TMax,gridMET,marszalek2022prediction,C
2,min_temp_c,TMin,gridMET,marszalek2022prediction,C
3,vpd_kpa,VPD,gridMET,joshi2023winter,kPa
4,potential_evapo_mmday,PEt,gridMET,qian2009statistical,mmday
5,palmer_drought_unitless,PDSI,gridMET,joshi2023winter,unitless
6,std_precip_evap_unitless,SPEI,gridMET,schierhorn2021machine,unitless
7,wind_speed_ms,WS,gridMET,marszalek2022prediction,ms
8,sr_wm2,SRad,gridMET,crane2018machine,Wm2
9,max_rh_perc,RHMax,gridMET,crane2018machine,percentage


In [6]:
file_list = os.listdir(raw_dir)
file_list = [x for x in file_list if x.endswith("xlsx")]
file_list[:3]

['Almira.xlsx', 'Bickleton.xlsx', 'Bonners Ferry.xlsx']

In [7]:
df = pd.read_excel(raw_dir + file_list[0])
df.head(3)

,date,doy,dap,Stage,Pr,TMax,TMin,VPD,PEt,AEt,...,SH,TAvg,RHAvg,DGDD,Dtr,PrDtr,GDD_C,WD,FDD,HDD
0,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,0.0,...,0.00299,-3.65,91.90,0.0,7.6,0.0,0.0,0,11.68,0.0
1,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,NaN,...,0.00302,-3.65,91.70,0.0,8.8,0.0,0.0,0,12.28,0.0
2,2001-01-03,3,2,Emergence,0.0,0.95,-5.55,0.04,0.1,NaN,...,0.00324,-2.30,88.15,0.0,6.5,0.0,0.0,0,9.78,0.0


In [8]:
### Check if all files share thew same columns

In [9]:
%%time

no_cols = 0
check = True

for a_file in file_list:
    df = df = pd.read_excel(raw_dir + a_file)
    
    if no_cols == 0:
        no_cols = df.shape[1]
        columns_set = set(df.columns)
    else:
        if df.shape[1] != no_cols:
            print (f"{a_file} column count is different")
        
        if columns_set != set(df.columns):
            print (f"{a_file} has different columns")
            check = False
            
if check:
    all_files = pd.DataFrame()
    # Example: read all CSV files in a folder
    for a_file in file_list:
        df = pd.read_excel(raw_dir + a_file)
        df['location'] = a_file.replace(".xlsx", "").replace(". ", "_").replace(" ", "_").lower()
        all_files = pd.concat([all_files, df], ignore_index=True)

CPU times: total: 49.9 s
Wall time: 50.8 s


In [10]:
sorted(df.columns)

['AEt',
 'DGDD',
 'Dtr',
 'FDD',
 'GDD_C',
 'HDD',
 'PDSI',
 'PEt',
 'Pr',
 'PrDtr',
 'RHAvg',
 'RHMax',
 'RHMin',
 'SH',
 'SM',
 'SPEI',
 'SRad',
 'Stage',
 'TAvg',
 'TMax',
 'TMin',
 'VPD',
 'WD',
 'WS',
 'dap',
 'date',
 'doy',
 'location']

In [11]:
sorted(all_files.columns)

['AEt',
 'DGDD',
 'Dtr',
 'FDD',
 'GDD_C',
 'HDD',
 'PDSI',
 'PEt',
 'Pr',
 'PrDtr',
 'RHAvg',
 'RHMax',
 'RHMin',
 'SH',
 'SM',
 'SPEI',
 'SRad',
 'Stage',
 'TAvg',
 'TMax',
 'TMin',
 'VPD',
 'WD',
 'WS',
 'dap',
 'date',
 'doy',
 'location']

In [12]:
sorted(all_files.location.unique())[:5]

['almira', 'bickleton', 'bonners_ferry', 'colfax', 'connell']

In [13]:
all_files.head(3)

,date,doy,dap,Stage,Pr,TMax,TMin,VPD,PEt,AEt,...,TAvg,RHAvg,DGDD,Dtr,PrDtr,GDD_C,WD,FDD,HDD,location
0,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,0.0,...,-3.65,91.90,0.0,7.6,0.0,0.0,0,11.68,0.0,almira
1,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,NaN,...,-3.65,91.70,0.0,8.8,0.0,0.0,0,12.28,0.0,almira
2,2001-01-03,3,2,Emergence,0.0,0.95,-5.55,0.04,0.1,NaN,...,-2.30,88.15,0.0,6.5,0.0,0.0,0,9.78,0.0,almira


In [14]:
fullName_units.shape

(28, 5)

In [15]:
fullName_units.head(2)

,variable_name,variable_short,source,reference,unit
0,precip_mmday,Pr,gridMET,marszalek2022prediction,mmday
1,max_temp_c,TMax,gridMET,marszalek2022prediction,C


In [16]:
fullName_units

,variable_name,variable_short,source,reference,unit
0,precip_mmday,Pr,gridMET,marszalek2022prediction,mmday
1,max_temp_c,TMax,gridMET,marszalek2022prediction,C
2,min_temp_c,TMin,gridMET,marszalek2022prediction,C
3,vpd_kpa,VPD,gridMET,joshi2023winter,kPa
4,potential_evapo_mmday,PEt,gridMET,qian2009statistical,mmday
5,palmer_drought_unitless,PDSI,gridMET,joshi2023winter,unitless
6,std_precip_evap_unitless,SPEI,gridMET,schierhorn2021machine,unitless
7,wind_speed_ms,WS,gridMET,marszalek2022prediction,ms
8,sr_wm2,SRad,gridMET,crane2018machine,Wm2
9,max_rh_perc,RHMax,gridMET,crane2018machine,percentage


In [17]:
# we do not need hour:minute:second:
all_files['date'] = pd.to_datetime(all_files['date']).apply(lambda x: x.date())
all_files.head(2)

,date,doy,dap,Stage,Pr,TMax,TMin,VPD,PEt,AEt,...,TAvg,RHAvg,DGDD,Dtr,PrDtr,GDD_C,WD,FDD,HDD,location
0,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,0.0,...,-3.65,91.9,0.0,7.6,0.0,0.0,0,11.68,0.0,almira
1,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,NaN,...,-3.65,91.7,0.0,8.8,0.0,0.0,0,12.28,0.0,almira


In [18]:
fullName_units.sort_values(by='variable_short', inplace=True)
fullName_units.reset_index(drop=True, inplace=True)
fullName_units.head(2)

,variable_name,variable_short,source,reference,unit
0,actual_evapo_mmmonth,AEt,TerraClimate,qian2009statistical,mmmonth
1,dailyGDD_diff_cday,DGDD,Derived Variable,li2022genetics,Cday


In [19]:
# Create a dictionary with 'variable_short' as the key and 'variable_name' as the value
variable_dict = fullName_units.set_index('variable_short')['variable_name'].to_dict()
variable_dict = {key.lower(): value for key, value in variable_dict.items()}

In [20]:
# A = all_files.copy()
# # Display the dictionary
# A.columns = A.columns.str.lower()
# A.rename(columns=variable_dict, inplace=True)
# sorted(A.columns)

In [21]:
# Display the dictionary
all_files.columns = all_files.columns.str.lower()
all_files.rename(columns=variable_dict, inplace=True)

In [22]:
all_files.head(3)

,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,actual_evapo_mmmonth,...,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,location
0,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,0.0,...,-3.65,91.90,0.0,7.6,0.0,0.0,0,11.68,0.0,almira
1,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,NaN,...,-3.65,91.70,0.0,8.8,0.0,0.0,0,12.28,0.0,almira
2,2001-01-03,3,2,Emergence,0.0,0.95,-5.55,0.04,0.1,NaN,...,-2.30,88.15,0.0,6.5,0.0,0.0,0,9.78,0.0,almira


### Read add varieties

In [23]:
%%time
planting_harvest_dates = pd.read_excel(data_base + "planting_harvest_dates.xlsx")
varieties_traits = pd.read_excel(data_base + "varieties_traits.xlsx")
planting_harvest_dates.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

planting_harvest_dates["location"] = \
          planting_harvest_dates["location"].str.replace(". ", "_").str.replace(" ", "_").str.lower()
planting_harvest_dates.head(2)

CPU times: total: 3.97 s
Wall time: 4.02 s


,year,location,planting_date,harvest_date,latitude,longitude
0,2002,almira,2002-04-08,2002-08-12,47.840633,-118.894615
1,2003,almira,2003-04-29,2003-08-20,47.840633,-118.894615


In [24]:
planting_harvest_dates[(planting_harvest_dates["location"] == "walla_walla") & \
                       (planting_harvest_dates["year"] == 2001)]["planting_date"].unique()

array([], dtype=object)

In [25]:
planting_harvest_dates[(planting_harvest_dates["location"] == "farmington") & \
                       (planting_harvest_dates["year"] == 2002)]

,year,location,planting_date,harvest_date,latitude,longitude
111,2002,farmington,2002-04-28,2002-09-04,47.081016,-117.05278


In [26]:
varieties_traits.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

varieties_traits.rename(columns={'irrigationstatus': 'irrigation_status'}, inplace=True)

varieties_traits["location"] = \
          varieties_traits["location"].str.replace(". ", "_").str.replace(" ", "_").str.lower()

varieties_traits.head(3)

,grain_yield,test_weight,species,heading_date,irrigation_status,tillage,plant_height,year,variety,protein_content,location
0,25.666667,58.866667,SWS,NaN,dryland,NaN,25.666667,2002,Alpowa,11.300000,almira
1,25.666667,57.433333,SWS,NaN,dryland,NaN,25.666667,2002,Alturas,11.766667,almira
2,26.333333,58.366667,HRS,NaN,dryland,NaN,26.333333,2002,BR2306,11.733333,almira


In [27]:
# Move location to the beginning of dataframe.
all_files = all_files[["location", *all_files.columns.drop("location")]]
all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,specific_humidity_kgkg,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,0.00299,-3.65,91.9,0.0,7.6,0.0,0.0,0,11.68,0.0
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,0.00302,-3.65,91.7,0.0,8.8,0.0,0.0,0,12.28,0.0


In [28]:
varieties_traits["irrigation_status"].unique()

array(['dryland', 'irrigated'], dtype=object)

In [29]:
varieties_traits.groupby(["irrigation_status"]).count()

,grain_yield,test_weight,species,heading_date,tillage,plant_height,year,variety,protein_content,location
irrigation_status,,,,,,,,,,
dryland,30874,27896,32326,23122,30937,30148,32326,32326,22557,32326
irrigated,32367,28941,36340,27991,36293,29109,36340,36340,6477,36340


In [30]:
varieties_traits.groupby(["irrigation_status"]).size()

irrigation_status
dryland      32326
irrigated    36340
dtype: int64

In [31]:
varieties_traits["grain_yield"].isna().sum() 

np.int64(5425)

In [32]:
varieties_traits[varieties_traits.irrigation_status=="irrigated"].location.unique()

array(['moses_lake', 'pasco', 'royal_slope', 'tulelake', 'klamath_falls',
       'la_grande', 'madras', 'north_willamette', 'ontario', 'athena',
       'parma', 'aberdeen', 'ashton', 'tetonia', 'idaho_falls', 'rupert'],
      dtype=object)

In [33]:
dryland_locs = list(varieties_traits[varieties_traits.irrigation_status=="dryland"].location.unique())
irrigated_locs = list(varieties_traits[varieties_traits.irrigation_status=="irrigated"].location.unique())

In [34]:
print([x for x in dryland_locs if ( x in irrigated_locs)])
print([x for x in irrigated_locs if ( x in dryland_locs)])

[]
[]


In [35]:
varieties_traits = varieties_traits[varieties_traits.irrigation_status=="dryland"].copy()
varieties_traits.reset_index(drop=True, inplace=True)
varieties_traits.head(3)
varieties_traits["irrigation_status"].unique()

# planting_harvest_dates.head(2)

array(['dryland'], dtype=object)

In [36]:
sorted(dryland_locs)[:5]

['almira', 'bickleton', 'bonners_ferry', 'colfax', 'connell']

In [37]:
sorted(irrigated_locs)[:5]

['aberdeen', 'ashton', 'athena', 'idaho_falls', 'klamath_falls']

In [38]:
# Move location to the beginning of dataframe.
varieties_traits.insert(0, "location", varieties_traits.pop("location"))
varieties_traits.head(2)

,location,grain_yield,test_weight,species,heading_date,irrigation_status,tillage,plant_height,year,variety,protein_content
0,almira,25.666667,58.866667,SWS,NaN,dryland,NaN,25.666667,2002,Alpowa,11.300000
1,almira,25.666667,57.433333,SWS,NaN,dryland,NaN,25.666667,2002,Alturas,11.766667


## Filter all_files and plant_dates: keep dryland only

In [39]:
# it appears weather data is available only for dryland
# as irrigated does not need them!
print (all_files.shape)
all_files = all_files[all_files.location.isin(varieties_traits["location"].unique())].copy()
print (all_files.shape)

(277200, 28)
(277200, 28)


In [40]:
all_files['date'][0]

datetime.date(2001, 1, 1)

In [41]:
all_files['date'] = pd.to_datetime(all_files['date'])
all_files["year"] = all_files['date'].dt.year
all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,-3.65,91.9,0.0,7.6,0.0,0.0,0,11.68,0.0,2001
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,-3.65,91.7,0.0,8.8,0.0,0.0,0,12.28,0.0,2001


In [42]:
sorted(planting_harvest_dates.location.unique())[:5]

['aberdeen', 'almira', 'ashton', 'athena', 'bickleton']

In [43]:
sorted(all_files.location.unique())[:5]

['almira', 'bickleton', 'bonners_ferry', 'colfax', 'connell']

In [44]:
# Pick drylands only
print (planting_harvest_dates.shape)
planting_harvest_dates = planting_harvest_dates[\
                                        planting_harvest_dates.location.isin(\
                                                            varieties_traits["location"].unique())].copy()
print (planting_harvest_dates.shape)

(517, 6)
(398, 6)


In [45]:
varieties_traits.head(2)

,location,grain_yield,test_weight,species,heading_date,irrigation_status,tillage,plant_height,year,variety,protein_content
0,almira,25.666667,58.866667,SWS,NaN,dryland,NaN,25.666667,2002,Alpowa,11.300000
1,almira,25.666667,57.433333,SWS,NaN,dryland,NaN,25.666667,2002,Alturas,11.766667


In [46]:
all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,-3.65,91.9,0.0,7.6,0.0,0.0,0,11.68,0.0,2001
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,-3.65,91.7,0.0,8.8,0.0,0.0,0,12.28,0.0,2001


In [47]:
planting_harvest_dates.head(2)

,year,location,planting_date,harvest_date,latitude,longitude
0,2002,almira,2002-04-08,2002-08-12,47.840633,-118.894615
1,2003,almira,2003-04-29,2003-08-20,47.840633,-118.894615


In [48]:
all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,-3.65,91.9,0.0,7.6,0.0,0.0,0,11.68,0.0,2001
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,-3.65,91.7,0.0,8.8,0.0,0.0,0,12.28,0.0,2001


In [49]:
all_files = pd.merge(all_files,
                     planting_harvest_dates[['year', 'location', 'planting_date', 'harvest_date']],
                     how="left", on=["location", "year"])

all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,0.0,7.6,0.0,0.0,0,11.68,0.0,2001,NaN,NaN
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,0.0,8.8,0.0,0.0,0,12.28,0.0,2001,NaN,NaN


In [50]:
planting_harvest_dates.head(2)

,year,location,planting_date,harvest_date,latitude,longitude
0,2002,almira,2002-04-08,2002-08-12,47.840633,-118.894615
1,2003,almira,2003-04-29,2003-08-20,47.840633,-118.894615


In [51]:
all_files[(all_files["location"]=="farmington") & (all_files["year"]==2002)].head(5)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
84365,farmington,2002-01-01,1,0,Emergence,0.0,2.15,-5.25,0.12,0.4,...,0.00,7.4,0.00000,0.00,0,9.48,0.0,2002,2002-04-28,2002-09-04
84366,farmington,2002-01-02,2,1,Emergence,2.7,3.15,-1.85,0.09,0.5,...,0.65,5.0,0.54000,0.65,1,6.08,0.0,2002,2002-04-28,2002-09-04
84367,farmington,2002-01-03,3,2,Emergence,3.2,2.75,-3.35,0.04,0.2,...,0.65,6.1,0.52459,0.65,1,7.58,0.0,2002,2002-04-28,2002-09-04
84368,farmington,2002-01-04,4,3,Emergence,0.0,2.15,-4.15,0.08,0.3,...,0.00,6.3,0.00000,0.65,0,8.38,0.0,2002,2002-04-28,2002-09-04
84369,farmington,2002-01-05,5,4,Emergence,0.0,3.15,-1.75,0.13,0.5,...,0.70,4.9,0.00000,1.35,0,5.98,0.0,2002,2002-04-28,2002-09-04


In [52]:
# We cannot do common years since we want preseason/pre-plant data:

# common_index = (all_files.set_index(["location", "year"]).index
#                 .intersection(planting_harvest_dates.set_index(["location", "year"]).index)
#                 .intersection(df3.set_index(["location", "year"]).index)
#                )

# common =  all_files.merge(planting_harvest_dates[['year', 'location', 'planting date', 'harvest date']], 
#                           on=["location", "year"], how="inner")

In [53]:
filename = raw_dir + "raw_all_locations_all_time_scales.sav"

export_ = {"wheat_all_locs_raw": all_files,
           "varieties_traits" : varieties_traits,
           "source_code": "read_organize_directories",
           "Author": "HN",
           "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
           "Note": "read and merged all files into 1 and changed column names for consistency"
          }

pickle.dump(export_, open(filename, 'wb'))

In [54]:
monthly_cols = ["actual_evapo_mmmonth", "soil_moisture_mm"]
fiveDay_cols = ["std_precip_evap_unitless", "palmer_drought_unitless"]

In [55]:
all_files.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,0.0,7.6,0.0,0.0,0,11.68,0.0,2001,NaN,NaN
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,0.0,8.8,0.0,0.0,0,12.28,0.0,2001,NaN,NaN


In [56]:
sorted(all_files.columns)

['actual_evapo_mmmonth',
 'avg_rh_perc',
 'avg_temp_c',
 'cum_gdd_cday',
 'dailyGDD_diff_cday',
 'dap',
 'date',
 'diurnal_temp_range_c',
 'doy',
 'freezing_dd_cday',
 'harvest_date',
 'heat_dd_cday',
 'location',
 'max_rh_perc',
 'max_temp_c',
 'min_rh_perc',
 'min_temp_c',
 'palmer_drought_unitless',
 'planting_date',
 'potential_evapo_mmday',
 'precip_dtr_mmdayc',
 'precip_mmday',
 'soil_moisture_mm',
 'specific_humidity_kgkg',
 'sr_wm2',
 'stage',
 'std_precip_evap_unitless',
 'vpd_kpa',
 'wet_day_frequency_days',
 'wind_speed_ms',
 'year']

In [57]:
static_cols = ['date', 'doy', 'location']

In [58]:
monthly_data = all_files[static_cols + monthly_cols].copy()
monthly_data.dropna(inplace=True)
monthly_data.reset_index(drop=True, inplace=True)
monthly_data.head(3)

,date,doy,location,actual_evapo_mmmonth,soil_moisture_mm
0,2001-01-01,1,almira,0.0,9.9
1,2001-02-01,32,almira,0.0,9.9
2,2001-03-01,60,almira,47.4,69.8


In [59]:
five_day_data = all_files[static_cols + fiveDay_cols].copy()
five_day_data.dropna(inplace=True)
five_day_data.reset_index(drop=True, inplace=True)
five_day_data.head(3)

,date,doy,location,std_precip_evap_unitless,palmer_drought_unitless
0,2001-01-05,5,almira,-0.32,-1.57
1,2001-01-10,10,almira,-1.13,-1.63
2,2001-01-15,15,almira,-0.90,-1.73


In [60]:
daily_df = all_files.copy()
daily_df.drop(columns=monthly_cols + fiveDay_cols, inplace=True)
daily_df.head(3)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,0.0,7.6,0.0,0.0,0,11.68,0.0,2001,NaN,NaN
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,0.0,8.8,0.0,0.0,0,12.28,0.0,2001,NaN,NaN
2,almira,2001-01-03,3,2,Emergence,0.0,0.95,-5.55,0.04,0.1,...,0.0,6.5,0.0,0.0,0,9.78,0.0,2001,NaN,NaN


In [61]:
filename = raw_dir + "raw_all_locations_separate_daily_monthly_fiveDay.sav"

export_ = {"wheat_all_locs_raw_daily": daily_df,
           "wheat_all_locs_raw_monthly": monthly_data,
           "wheat_all_locs_raw_fiveDay": five_day_data,
           "varieties_traits":varieties_traits,
           "source_code": "read_organize_directories",
           "Author": "HN",
           "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
           "Note": "read and merged all files into 1 and changed column names for consistency"
          }

pickle.dump(export_, open(filename, 'wb'))